Version: 02.14.2023

# Lab 7.1: Implementing a Multilingual Solution

In this lab, you will learn how to use three different AWS services to create a solution that translates an audio file into text in a different language.

## Introducing the business scenario

In this lab, you play the role of a machine learning (ML) developer working for a media company that translates videos between multiple languages.  

## Lab steps

To complete this lab, you will follow these steps:

1. [Amazon Transcribe example](#1.-Amazon-Transcribe-example)
2. [Amazon Translate example](#2.-Amazon-Translate-example)
3. [Amazon Polly example](#3.-Amazon-Polly-example)
4. [Challenge exercise](#4.-Challenge-exercise)

## Submitting your work

1. In the lab console, choose **Submit** to record your progress and when prompted, choose **Yes**.

1. If the results don't display after a couple of minutes, return to the top of the lab instructions and choose **Grades**.

    **Tip:** You can submit your work multiple times. After you change your work, choose **Submit** again. Your last submission is what will be recorded for this lab.

1. To find detailed feedback on your work, choose **Details** followed by **View Submission Report**.
    

## 1. Amazon Transcribe example
([Go to top](#Lab-7.1:-Implementing-a-Multilingual-Solution))

In this section, you will use the AWS SDK for Python (boto3) client to call Amazon Transcribe and convert an audio file into text. After running the example, you can go to the [Amazon Transcribe](https://console.aws.amazon.com/transcribe/home?region=us-east-1) console to see the transcription.

It will take a few minutes for the transcription to complete.

In [1]:
import uuid
import json
import boto3
from time import sleep

bucket = 'c96181a2162112l4832180t1w893320015925-labbucket-cvsmrp1v7qb1'
database_access_role_arn = 'arn:aws:iam::893320015925:role/service-role/c96181a2162112l4832180t1w-ComprehendDataAccessRole-TKONH7MA2DWL'
translate_access_role_arn = 'arn:aws:iam::893320015925:role/c96181a2162112l4832180t1w8933200-TranslateDemoRole-25HRMEFTBN14'


transcribe_client = boto3.client("transcribe")

The test.wav sample file is located in the **/s3** folder. The file contains the audio phrase "Test. Hello, hello, hello. This is a test. Test, test, test."

In [2]:
#Create input parameters for job_name and job_uri
media_input_uri = f's3://{bucket}/lab71/transcribe-sample/test.wav'

First, create the transcription job with the test.wav file as the input. Note that you need to specify an output location.

In [3]:
#create the transcription job
job_uuid = uuid.uuid1()
transcribe_job_name = f"transcribe-job-{job_uuid}"
transcribe_output_filename = 'transcribe_output.txt'

response = transcribe_client.start_transcription_job(
    TranscriptionJobName=transcribe_job_name,
    Media={'MediaFileUri': media_input_uri},
    MediaFormat='wav',
    LanguageCode='en-US',
    OutputBucketName=bucket,
    OutputKey=transcribe_output_filename
)

Wait until the job completes.

In [4]:
job=None
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName = transcribe_job_name)
    if job['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
        break
    print('.', end='')
    sleep(20)
        
print(job['TranscriptionJob']['TranscriptionJobStatus'])

..COMPLETED


If the output from the previous cell is *COMPLETED*, then proceed. Otherwise, correct the error and retry the previous cell.

To retrieve the output file, use the results from the `get_transcription_job` method call.

In [5]:
transcription_file = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
print(transcription_file)

https://s3.us-east-1.amazonaws.com/c96181a2162112l4832180t1w893320015925-labbucket-cvsmrp1v7qb1/transcribe_output.txt


To download the file from Amazon Simple Storage Service (Amazon S3), use the S3 client.

In [6]:
s3_client = boto3.client('s3')
with open(transcribe_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, transcribe_output_filename, f)

Open the file and read the contents into a JSON object.

In [7]:
with open(transcribe_output_filename) as f:
  data = json.load(f)

In [8]:
data

{'jobName': 'transcribe-job-1382c006-5bcb-11ee-a6c3-0ec82977866d',
 'accountId': '893320015925',
 'results': {'transcripts': [{'transcript': 'Test. Hello. Hello. Hello. This is a test test test test.'}],
  'items': [{'start_time': '0.009',
    'end_time': '0.49',
    'alternatives': [{'confidence': '0.995', 'content': 'Test'}],
    'type': 'pronunciation'},
   {'alternatives': [{'confidence': '0.0', 'content': '.'}],
    'type': 'punctuation'},
   {'start_time': '0.589',
    'end_time': '1.259',
    'alternatives': [{'confidence': '0.998', 'content': 'Hello'}],
    'type': 'pronunciation'},
   {'alternatives': [{'confidence': '0.0', 'content': '.'}],
    'type': 'punctuation'},
   {'start_time': '1.269',
    'end_time': '1.74',
    'alternatives': [{'confidence': '0.997', 'content': 'Hello'}],
    'type': 'pronunciation'},
   {'alternatives': [{'confidence': '0.0', 'content': '.'}],
    'type': 'punctuation'},
   {'start_time': '1.75',
    'end_time': '2.46',
    'alternatives': [{'con

To get the actual transcription, run the following cell.

In [9]:
data['results']['transcripts'][0]['transcript']

'Test. Hello. Hello. Hello. This is a test test test test.'

## 2. Amazon Translate example

([Go to top](#Lab-7.1:-Implementing-a-Multilingual-Solution))

In this section, you will use the AWS SDK for Python (boto3) client to call Amazon Translate and convert a text file from English to Spanish. After running the example, you can go to the [Amazon Translate](https://console.aws.amazon.com/translate/home?region=us-east-1#batch-translation) console to see the translation.

The translation and details about the job are located in the **Batch translation** section. The text file containing the translation will be in your S3 bucket. The bucket will also have a **details** folder that contains a JSON file with details about the translation, such as the source and target languages.

First, create the translation job. The input and output locations are required.

Note that Amazon Translate can translate the same text into multiple target languages. In this example, you will use Spanish, for which the language code is `es`.

In [10]:
import uuid

translate_client = boto3.client(service_name='translate')

input_data = f's3://{bucket}/lab71/translate-sample'
output_data = f's3://{bucket}'

job_uuid = uuid.uuid1()
translate_job_name = f"translate-job-{job_uuid}"
translate_job_submission = translate_client.start_text_translation_job(
    JobName=translate_job_name,
    InputDataConfig={'S3Uri': input_data, 'ContentType':'text/plain'},
    OutputDataConfig={'S3Uri': output_data},
    DataAccessRoleArn=translate_access_role_arn,
    SourceLanguageCode='en',
    TargetLanguageCodes=['es']
)
translate_job_id = translate_job_submission['JobId']

Use the job ID from the previous cell to get the status. Wait for the job to complete.

Note that the job will take a few minutes to complete.

In [11]:
while True:
    translate_job = translate_client.describe_text_translation_job(JobId=translate_job_id)
    if translate_job['TextTranslationJobProperties']['JobStatus'] in ['COMPLETED','FAILED']:
        break
    sleep(20)
    print('.', end='')

print(translate_job['TextTranslationJobProperties']['JobStatus'])

.............................................COMPLETED


If the output from the previous cell is *COMPLETED*, then proceed. Otherwise, correct the error and retry the previous cell.

The format of the output folder is created from the account number and job ID. The following cell creates a path with this information.

In [12]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
translate_output_path = f'{account_id}-TranslateText-{translate_job_id}/'

Amazon Translate outputs several files. You are interested in the .txt file, which contains the results from the translation. The following cell will download the .txt file.

In [13]:
s3_resource = boto3.resource('s3')
my_bucket = s3_resource.Bucket(bucket)

for my_bucket_object in my_bucket.objects.filter(Prefix=translate_output_path):
    file=my_bucket_object.key
    if file.endswith('txt'):
        file = file.lstrip(translate_output_path)
        file = file.lstrip('/')
        print(file)
        with open(file, 'wb') as f:
            s3_client.download_fileobj(bucket, my_bucket_object.key, f)

.test.txt


## 3. Amazon Polly example

([Go to top](#Lab-7.1:-Implementing-a-Multilingual-Solution))

In this section, you will use the AWS SDK for Python (boto3) client to call Amazon Polly and create a vocalization of a text file in Spanish.

After you run the cell, open your S3 bucket to see the output. The output is an .mp3 file with a long string as the file name. You can open the file and hear the Lucia voice saying "Prueba de prueba, este es una prueba."


In [14]:
polly_client = boto3.client('polly')

itemname = 'lab71/polly-sample/es.test.txt'
obj = s3_resource.Object(bucket, itemname )
body = obj.get()['Body'].read().decode('utf-8')

response = polly_client.start_speech_synthesis_task(
    Engine='standard',
    OutputFormat='mp3',
    OutputS3BucketName=bucket,
    Text=body,
    VoiceId='Lucia'
) 


Extract the task ID from the response.

In [15]:
task_id = response['SynthesisTask']['TaskId']

Use this task ID to check whether the job is complete.

In [16]:
while True:
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    if polly_job['SynthesisTask']['TaskStatus'] in ['completed','failed']:
        break
    sleep(20)
    print('.', end='')

print(polly_job['SynthesisTask']['TaskStatus'])

.completed


If the output from the previous cell is *completed*, then proceed. Otherwise, correct the error and retry the previous cell.

To download the results, run the following cell.

In [17]:
s3_client = boto3.client('s3')
polly_output_filename = f'{task_id}.mp3'
with open(polly_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, polly_output_filename, f)

## 4. Challenge exercise

([Go to top](#Lab-7.1:-Implementing-a-Multilingual-Solution))
    
Your challenge is to create a translated audio file from a video that has an English audio channel. You can use the code from the previous three examples as a template for your solution.

The video for the challenge is in your S3 bucket in the `lab71/challenge` folder. The video file is named sample.mp4. This file is also available in the `/s3` folder in this notebook instance.

# Congratulations!

You have completed this lab, and you can now end the lab by following the lab guide instructions.

*©2023 Amazon Web Services, Inc. or its affiliates. All rights reserved. This work may not be reproduced or redistributed, in whole or in part, without prior written permission from Amazon Web Services, Inc. Commercial copying, lending, or selling is prohibited. All trademarks are the property of their owners.*

In [30]:
#Create input parameters for job_name and job_uri
bucket = 'c96181a2162112l4832180t1w893320015925-labbucket-cvsmrp1v7qb1'
media_input_uri = f's3://{bucket}/lab71/challenge/sample.mp4'

In [31]:
#create the transcription job
job_uuid = uuid.uuid1()
transcribe_job_name1 = f"transcribe-job1-{job_uuid}"
transcribe_output_filename = 'transcribe_output.txt'


response = transcribe_client.start_transcription_job(
    TranscriptionJobName=transcribe_job_name1,
    Media={'MediaFileUri': media_input_uri},
    MediaFormat='mp4',
    LanguageCode='en-US',
    OutputBucketName=bucket,
    OutputKey=transcribe_output_filename
)

In [32]:
job=None
while True:
    job = transcribe_client.get_transcription_job(TranscriptionJobName = transcribe_job_name1)
    if job['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED','FAILED']:
        break
    print('.', end='')
    sleep(20)
        
print(job['TranscriptionJob']['TranscriptionJobStatus'])

....COMPLETED


In [33]:
transcription_file = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
print(transcription_file)

https://s3.us-east-1.amazonaws.com/c96181a2162112l4832180t1w893320015925-labbucket-cvsmrp1v7qb1/transcribe_output.txt


In [34]:
s3_client = boto3.client('s3')
with open(transcribe_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, transcribe_output_filename, f)

In [35]:
with open(transcribe_output_filename) as f:
    data = json.load(f)

In [37]:
data

{'jobName': 'transcribe-job1-7f0e8ffa-5bce-11ee-a6c3-0ec82977866d',
 'accountId': '893320015925',
 'results': {'transcripts': [{'transcript': "Hi and welcome to Amazon Academy of Machine Learning Foundations in this module, you'll learn about the course objectives, various job roles in the machine learning domain and where you can go to learn more about machine learning. After completing this module, you should be able to identify course prerequisites and objectives indicate the role of the data scientist in business and identify resources for further learning. We're now going to look at the prerequisites for taking this course. Before you take this course, we recommend that you first complete Aws Academy Cloud Foundations. You should also have some general technical knowledge of it including foundational computer literacy skills like basic computer concepts, email file management and a good understanding of the internet. We also recommend that you have intermediate skills with Python 

In [38]:
data['results']['transcripts'][0]['transcript']

"Hi and welcome to Amazon Academy of Machine Learning Foundations in this module, you'll learn about the course objectives, various job roles in the machine learning domain and where you can go to learn more about machine learning. After completing this module, you should be able to identify course prerequisites and objectives indicate the role of the data scientist in business and identify resources for further learning. We're now going to look at the prerequisites for taking this course. Before you take this course, we recommend that you first complete Aws Academy Cloud Foundations. You should also have some general technical knowledge of it including foundational computer literacy skills like basic computer concepts, email file management and a good understanding of the internet. We also recommend that you have intermediate skills with Python programming and a general knowledge of applied statistics. Finally, general business knowledge is important for this course. This includes ins

In [53]:
polly_client = boto3.client('polly')

itemname = 'lab71/challenge/transcribe_output.txt'
bucket = 'c96181a2162112l4832180t1w893320015925-labbucket-cvsmrp1v7qb1'

In [62]:
transcribe_output_filename

'transcribe_output.txt'

In [66]:

try:
    # Retrieve the object from S3
    response = s3_client.get_object(Bucket=bucket, Key=transcribe_output_filename)
    
    # Read the object's content as UTF-8 text
    text = response['Body'].read().decode('utf-8')
    
    # Split the text into segments (e.g., every 3,000 characters)
    text_segments = [text[i:i + 3000] for i in range(0, len(text), 3000)]
    
    # Start a Polly synthesis task for each segment
    for i, segment in enumerate(text_segments):
        response = polly_client.start_speech_synthesis_task(
            Engine='standard',
            OutputFormat='mp3',
            OutputS3BucketName=bucket,
            Text=segment,
            VoiceId='Lucia'
        )

        print(f"Speech synthesis task {i+1} started successfully.")

except Exception as e:
    print(f"An error occurred: {str(e)}")


Speech synthesis task 1 started successfully.
Speech synthesis task 2 started successfully.
Speech synthesis task 3 started successfully.
Speech synthesis task 4 started successfully.
Speech synthesis task 5 started successfully.
Speech synthesis task 6 started successfully.
Speech synthesis task 7 started successfully.
Speech synthesis task 8 started successfully.
Speech synthesis task 9 started successfully.
Speech synthesis task 10 started successfully.
Speech synthesis task 11 started successfully.
Speech synthesis task 12 started successfully.
Speech synthesis task 13 started successfully.
Speech synthesis task 14 started successfully.
Speech synthesis task 15 started successfully.
Speech synthesis task 16 started successfully.
Speech synthesis task 17 started successfully.
Speech synthesis task 18 started successfully.
Speech synthesis task 19 started successfully.
Speech synthesis task 20 started successfully.
Speech synthesis task 21 started successfully.
Speech synthesis task 

In [67]:
task_id = response['SynthesisTask']['TaskId']

In [68]:
while True:
    polly_job = polly_client.get_speech_synthesis_task(TaskId=task_id)
    if polly_job['SynthesisTask']['TaskStatus'] in ['completed','failed']:
        break
    sleep(20)
    print('.', end='')

print(polly_job['SynthesisTask']['TaskStatus'])

completed


In [69]:
s3_client = boto3.client('s3')
polly_output_filename = f'{task_id}.mp3'
with open(polly_output_filename, 'wb') as f:
    s3_client.download_fileobj(bucket, polly_output_filename, f)